In [51]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [39]:
city_weather_df = pd.read_csv("Output/weather_df.csv")
city_weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,mataura,75,NZ,1633829151,40,-46.1927,168.8643,64.47,4.59
1,1,bluff,58,NZ,1633828853,57,-46.6000,168.3333,64.80,3.24
2,2,barrow,90,US,1633829152,80,71.2906,-156.7887,30.22,4.61
3,3,coquimbo,0,CL,1633829153,71,-29.9533,-71.3436,55.13,5.75
4,4,hasaki,31,JP,1633829153,76,35.7333,140.8333,73.69,6.71
...,...,...,...,...,...,...,...,...,...,...
566,566,camden,100,US,1633829187,88,39.7835,-74.9663,66.88,1.01
567,567,adrar,55,MR,1633829351,16,20.5022,-10.0711,84.20,8.90
568,568,tshikapa,99,CD,1633829351,58,-6.4167,20.8000,77.58,2.46
569,569,raton,1,US,1633829267,27,36.9034,-104.4391,63.21,16.11


In [40]:
#Use the Lat and Lng as locations
locations = city_weather_df[["Lat","Lng"]]

#Add Heatmap layer to map
humidity = city_weather_df["Humidity"]

In [41]:
#Plot heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [46]:
#Narrow down the cities to fit weather conditions
#Drop any rows will null values
smaller_city_df = city_weather_df.loc[(city_weather_df["Wind Speed"] <= 10) & (city_weather_df["Cloudiness"]== 0) & \
                  (city_weather_df["Max Temp"] >= 70) & (city_weather_df["Max Temp"] <= 80)].dropna()
smaller_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
35,35,morondava,0,MG,1633829165,85,-20.2833,44.2833,73.98,3.85
109,109,jalu,0,LY,1633829193,33,29.0331,21.5482,74.57,9.53
114,114,borba,0,BR,1633829195,71,-4.3878,-59.5939,79.07,1.10
147,147,tura,0,IN,1633829206,86,25.5198,90.2201,75.38,2.57
169,169,poum,0,NC,1633829215,65,-20.2333,164.0167,74.34,9.26
170,170,gat,0,IL,1633829215,74,31.6100,34.7642,72.39,5.14
173,173,nautanwa,0,IN,1633829216,82,27.4333,83.4167,74.05,2.64
222,222,avera,0,US,1633829234,86,33.1940,-82.5271,70.72,6.55
223,223,jacareacanga,0,BR,1633829235,72,-6.2222,-57.7528,78.53,1.39
310,310,phalombe,0,MW,1633829265,28,-15.8064,35.6507,70.11,1.03


In [47]:
#Store into variable named hotel_df
hotel_df = smaller_city_df.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
35,morondava,MG,-20.2833,44.2833,
109,jalu,LY,29.0331,21.5482,
114,borba,BR,-4.3878,-59.5939,
147,tura,IN,25.5198,90.2201,
169,poum,NC,-20.2333,164.0167,
170,gat,IL,31.6100,34.7642,
173,nautanwa,IN,27.4333,83.4167,
222,avera,US,33.1940,-82.5271,
223,jacareacanga,BR,-6.2222,-57.7528,
310,phalombe,MW,-15.8064,35.6507,


In [48]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [52]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------Ending Search")

Retrieving Results for Index 35: morondava.
Closest hotel in morondava is Palissandre Cote Ouest resort & SPA.
--------------------Ending Search
Retrieving Results for Index 109: jalu.
Closest hotel in jalu is Thuraya Tourist Hotel.
--------------------Ending Search
Retrieving Results for Index 114: borba.
Closest hotel in borba is Hotel da Amazônia.
--------------------Ending Search
Retrieving Results for Index 147: tura.
Closest hotel in tura is Hotel C&C.
--------------------Ending Search
Retrieving Results for Index 169: poum.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 170: gat.
Closest hotel in gat is Desert Gat.
--------------------Ending Search
Retrieving Results for Index 173: nautanwa.
Closest hotel in nautanwa is Hotel Mamata.
--------------------Ending Search
Retrieving Results for Index 222: avera.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index

In [53]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
35,morondava,MG,-20.2833,44.2833,Palissandre Cote Ouest resort & SPA
109,jalu,LY,29.0331,21.5482,Thuraya Tourist Hotel
114,borba,BR,-4.3878,-59.5939,Hotel da Amazônia
147,tura,IN,25.5198,90.2201,Hotel C&C
169,poum,NC,-20.2333,164.0167,
170,gat,IL,31.6100,34.7642,Desert Gat
173,nautanwa,IN,27.4333,83.4167,Hotel Mamata
222,avera,US,33.1940,-82.5271,
223,jacareacanga,BR,-6.2222,-57.7528,HOTEL VIANA
310,phalombe,MW,-15.8064,35.6507,


In [54]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [55]:
# Plot markers on top of the heatmap
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))